<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
# enddate = date.today()
# startdate = enddate - relativedelta(years=4)

# df = yf.download('HINDZINC.NS', start=startdate, end=enddate)


In [4]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# last_quote = ticker_yahoo.fast_info['last_price']
# MA200 = ticker_yahoo.fast_info['two_hundred_day_average']
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)
#print(tickers)

In [ ]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [6]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [ ]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)
#ticker_list = ticker_list.loc[(ticker_list['Last_price'] < ticker_list['ATH'] * 0.95) & (ticker_list['Last_price'] > ticker_list['ATH'] * 0.9) & (ticker_list['Last_price'] > ticker_list['two_EMA'])]

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)
ticker_list
#ticker_list.to_csv('Watchlist.csv', index=False)


In [58]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]
#tickers
df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  101 of 101 completed


In [88]:
# df.ta.supertrend(high='High', low='Low', close='Close', length=10, multiplier=1, append=True)
# df.to_csv('data1.csv')
#tickers
help(ta.supertrend)

Help on function supertrend in module pandas_ta.overlap.supertrend:

supertrend(high, low, close, length=None, multiplier=None, offset=None, **kwargs)
    Supertrend (supertrend)
    
    Supertrend is an overlap indicator. It is used to help identify trend
    direction, setting stop loss, identify support and resistance, and/or
    generate buy & sell signals.
    
    Sources:
        http://www.freebsensetips.com/blog/detail/7/What-is-supertrend-indicator-its-calculation
    
    Calculation:
        Default Inputs:
            length=7, multiplier=3.0
        Default Direction:
            Set to +1 or bullish trend at start
    
        MID = multiplier * ATR
        LOWERBAND = HL2 - MID
        UPPERBAND = HL2 + MID
    
        if UPPERBAND[i] < FINAL_UPPERBAND[i-1] and close[i-1] > FINAL_UPPERBAND[i-1]:
            FINAL_UPPERBAND[i] = UPPERBAND[i]
        else:
            FINAL_UPPERBAND[i] = FINAL_UPPERBAND[i-1])
    
        if LOWERBAND[i] > FINAL_LOWERBAND[i-1] and clos

In [87]:
technicals =['supertrend10','supertrend11','supertrend12']
pd.set_option('display.max_rows', 10)

# for t in technicals:
#   print(t[10:])
  

#df[('Close', 'ABBOTINDIA.NS' )]

for ticker in tickers:
  #df[('EMA_200',ticker)] = ta.ema(df.loc[:,('Close', ticker)], length=200)
  for t in technicals:
    if t[:10] == 'supertrend':
      if int(t[10:]) == 10:
        df[(t,ticker)] = ta.supertrend(close=df[('Close', ticker)], high=df[('High', ticker)], low=df[('Low', ticker)], length=10, multiplier=1)
        #df.ta.supertrend(close=df[('Close', ticker)], high=df[('High', ticker)], low=df[('Low', ticker)], length=10, multiplier=1, append= True)
      elif int(t[10:]) == 11:
        df[(t,ticker)] = ta.supertrend(close=df[('Close', ticker)], high=df[('High', ticker)], low=df[('Low', ticker)], length=11, multiplier=2)
      elif int(t[10:]) == 12:
        df[(t,ticker)] = ta.supertrend(close=df[('Close', ticker)], high=df[('High', ticker)], low=df[('Low', ticker)], length=12, multiplier=3)
    


KeyError: ignored